In [1]:
from dotenv import load_dotenv
import os

# Langchain
from langchain_community.graphs import Neo4jGraph
import chunking
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_ENDPOINT = os.getenv('OPENAI_BASE_URL') + '/embeddings'


kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [14]:
cypher = """ 
// Person Nodes
CREATE (napoleon:Person {
    name: "Napoleon Bonaparte"
})


CREATE (talleyrand:Person {
    name: "Charles-Maurice de Talleyrand"
})

// Event node
CREATE (waterloo:Event {
    name: "Battle of Waterloo"
})


// waterloo nodes
CREATE (waterlooGeneral:General_info {
    chunk_info: "General information",
    battleDate: "1815-06-18",
    location: "Waterloo, Belgium",
    outcome: "Decisive defeat for Napoleon",
    commander: "Duke of Wellington"
})
CREATE (waterlooReason:Reason {
    chunk_info: "Reason",
    cause: "Napoleon's return from exile",
    strategicMistake: "Failure to unite forces before battle",
    politicalImpact: "End of the Napoleonic Wars"
})

CREATE (waterlooCombatant:Combatant {
    chunk_info: "Combatant",
    frenchCommander: "Napoleon Bonaparte",
    alliedCommander: "Duke of Wellington",
    prussianCommander: "Gebhard Leberecht von Blücher",
    mainForces: "French Army, British Army, Prussian Army"
})

CREATE (waterlooConsequence:Consequence {
    chunk_info: "Consequence",
    immediateResult: "Napoleon's second abdication",
    longTermImpact: "Restoration of the Bourbon monarchy",
    geopoliticalEffect: "Redrawing of European borders"
})




// Career Nodes
CREATE (napoleonGeneral:General_info {
    chunk_info: "General Information",
    birthDate: "1769-08-15",
    deathDate: "1821-05-05",
    nationality: "French",
    knownFor: "Military and political leader"
})

CREATE (talleyrandGeneral:General_info {
    chunk_info: "General Information",
    birthDate: "1754-02-02",
    deathDate: "1838-05-17",
    nationality: "French",
    knownFor: "Diplomat and statesman"
})


CREATE (napoleonCareer:Career {
    position: "Emperor",
    period: "1804-1814",
    chunk_info: "Career"
})

CREATE (talleyrandCareer:Career {
    position: "Foreign Minister",
    period: "1799-1807",
    chunk_info: "Career"
})

// Death Nodes
CREATE (napoleonDeath:Death {
    date: "1821-05-05",
    location: "Longwood, Saint Helena",
    chunk_info: "Death"
})

CREATE (talleyrandDeath:Death {
    date: "1838-05-17",
    location: "Paris, France",
    chunk_info: "Death"
})
// Create relationships for career and death information
CREATE (napoleon)-[:HAS_Career_INFO]->(napoleonCareer)
CREATE (napoleon)-[:HAS_Death_INFO]->(napoleonDeath)
CREATE (napoleon)-[:HAS_General_INFO]->(napoleonGeneral)

CREATE (talleyrand)-[:HAS_Career_INFO]->(talleyrandCareer)
CREATE (talleyrand)-[:HAS_Death_INFO]->(talleyrandDeath)
CREATE (talleyrand)-[:HAS_General_INFO]->(talleyrandGeneral)


// Create relationships between Person nodes
CREATE (napoleon)-[:RELATED_TO]->(talleyrand)
CREATE (talleyrand)-[:RELATED_TO]->(napoleon)
// Create relationships between Person nodes and Event
CREATE (napoleon)-[:RELATED_TO]->(waterloo)
CREATE (talleyrand)-[:RELATED_TO]->(waterloo)


// Create relationships between waerloo nodes
CREATE (waterloo)-[:HAS_General_INFO]->(waterlooGeneral)
CREATE (waterloo)-[:HAS_Reason_INFO]->(waterlooReason)
CREATE (waterloo)-[:HAS_Combatant_INFO]->(waterlooCombatant)
CREATE (waterloo)-[:HAS_Consequence_INFO]->(waterlooConsequence)
"""



kg.query(cypher)

[]

In [15]:
# Connect napoleon Career node to each chunk of napoleon career
cypher = """ 
MATCH (napoleonCareer:Career {position: "Emperor"}), (careerChunks:Napoleon_Chunk)
  WHERE napoleonCareer.chunk_info = careerChunks.formItem 
    WITH napoleonCareer, careerChunks
MERGE (napoleonCareer)-[r:HAS_Chunk_INFO]->(careerChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 34}]

In [16]:
# Connect napoleon death node to each chunk of napoleon death
cypher = """ 
MATCH (napoleonDeath:Death {location: "Longwood, Saint Helena"}), (deathChunks:Napoleon_Chunk)
  WHERE napoleonDeath.chunk_info = deathChunks.formItem 
    WITH napoleonDeath, deathChunks
MERGE (napoleonDeath)-[r:HAS_Chunk_INFO]->(deathChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 8}]

In [17]:
# Connect napoleon person node to each chunk of napoleon general information
cypher = """ 
MATCH (napoleonGeneral:General_info {knownFor: "Military and political leader"}), (generalChunks:Napoleon_Chunk)
  WHERE napoleonGeneral.chunk_info = generalChunks.formItem
    WITH napoleonGeneral, generalChunks
MERGE (napoleonGeneral)-[r:HAS_Chunk_INFO]->(generalChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 18}]

In [18]:
# Connect Talleyrand Career node to each chunk of Talleyrand career
cypher = """ 
MATCH (TalleyrandCareer:Career {position: "Foreign Minister"}), (careerChunks:Talleyrand_Chunk)
  WHERE TalleyrandCareer.chunk_info = careerChunks.formItem 
    WITH TalleyrandCareer, careerChunks
MERGE (TalleyrandCareer)-[r:HAS_Chunk_INFO]->(careerChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 13}]

In [19]:
# Connect Talleyrand Career node to each chunk of Talleyrand career
cypher = """ 
MATCH (TalleyrandDeath:Death {location: "Paris, France"}), (careerChunks:Talleyrand_Chunk)
  WHERE TalleyrandDeath.chunk_info = careerChunks.formItem 
    WITH TalleyrandDeath, careerChunks
MERGE (TalleyrandDeath)-[r:HAS_Chunk_INFO]->(careerChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 1}]

In [20]:
# Connect Talleyrand general node to each chunk of Talleyrand general information
cypher = """ 
MATCH (TalleyrandGeneral:General_info {knownFor: "Diplomat and statesman"}), (generalChunks:Talleyrand_Chunk)
  WHERE TalleyrandGeneral.chunk_info = generalChunks.formItem
    WITH TalleyrandGeneral, generalChunks
MERGE (TalleyrandGeneral)-[r:HAS_Chunk_INFO]->(generalChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 12}]

In [21]:
# Connect waterloo general node to each chunk of waterloo general information
cypher = """ 
MATCH (waterlooGeneral:General_info {chunk_info: "General information"}), (waterlooChunks:Waterloo_Chunk)
    WHERE waterlooGeneral.chunk_info = waterlooChunks.formItem
    WITH waterlooGeneral, waterlooChunks
MERGE (waterlooGeneral)-[r:HAS_Chunk_INFO]->(waterlooChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 5}]

In [22]:
# Connect waterloo Reason node to each chunk of waterloo Reason information
cypher = """ 
MATCH (waterlooReason:Reason {chunk_info: "Reason"}), (waterlooChunks:Waterloo_Chunk)
    WHERE waterlooReason.chunk_info = waterlooChunks.formItem
    WITH waterlooReason, waterlooChunks
MERGE (waterlooReason)-[r:HAS_Chunk_INFO]->(waterlooChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 5}]

In [23]:
# Connect waterloo Combatant node to each chunk of waterloo Combatant information
cypher = """ 
MATCH (waterlooCombatant:Combatant {chunk_info: "Combatant"}), (waterlooChunks:Waterloo_Chunk)
    WHERE waterlooCombatant.chunk_info = waterlooChunks.formItem
    WITH waterlooCombatant, waterlooChunks
MERGE (waterlooCombatant)-[r:HAS_Chunk_INFO]->(waterlooChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 23}]

In [24]:
# Connect waterloo Consequence node to each chunk of waterloo Consequence information
cypher = """ 
MATCH (waterlooConsequence:Consequence {chunk_info: "Consequence"}), (waterlooChunks:Waterloo_Chunk)
    WHERE waterlooConsequence.chunk_info = waterlooChunks.formItem
    WITH waterlooConsequence, waterlooChunks
MERGE (waterlooConsequence)-[r:HAS_Chunk_INFO]->(waterlooChunks)
RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 28}]